In [1]:
import sys
import os
from pyltp import Segmentor, Postagger, Parser, NamedEntityRecognizer# 需装Python后再下载模型，此次运行的模型版本为3.4.0

In [2]:
print("正在加载LTP模型... ...")
# Set your own model path
MODELDIR="E:\pyltp_model\ltp_data_v3.4.0"

segmentor = Segmentor()
# segmentor.load(os.path.join(MODELDIR, "cws.model"))
cws_model_path=os.path.join(MODELDIR, "cws.model")
segmentor.load_with_lexicon(cws_model_path, './dict.txt') #请把字典文件放进当前文件夹
               
postagger = Postagger()
postagger.load(os.path.join(MODELDIR, "pos.model"))

parser = Parser()
parser.load(os.path.join(MODELDIR, "parser.model"))

recognizer = NamedEntityRecognizer()
recognizer.load(os.path.join(MODELDIR, "ner.model"))

#labeller = SementicRoleLabeller()
#labeller.load(os.path.join(MODELDIR, "srl/"))

print("加载模型完毕。")

正在加载LTP模型... ...
加载模型完毕。


In [131]:
sentence='小明是足球运动员，也是杰出的上海人，也是优秀的中国人'

In [132]:
    # 切割句子
    words = segmentor.segment(sentence)
    # 词性标注
    postags = postagger.postag(words)
    # 命名实体识别
    netags = recognizer.recognize(words, postags)
    # 依存句法分析，其中已有模型可将句子中的主谓宾等结构抽取出来
    arcs = parser.parse(words, postags)

    child_dict_list = build_parse_child_dict(words, postags, arcs)

是 [0]


In [133]:
i=1
for word in words:
    print(str(i)+word)
    i+=1

1小明
2是
3足球
4运动员
5，
6也
7是
8杰出
9的
10上海
11人
12，
13也
14是
15优秀
16的
17中国
18人


In [134]:
i=1
for postag in postags:
    print(str(i)+postag)
    i+=1

1nh
2v
3n
4n
5wp
6d
7v
8a
9u
10ns
11n
12wp
13d
14v
15a
16u
17ns
18n


In [135]:
i=1
for arc in arcs:
    print(str(i)+'->'+str(arc.head)+'：'+arc.relation)
    i+=1

1->2：SBV
2->0：HED
3->4：ATT
4->2：VOB
5->2：WP
6->7：ADV
7->2：COO
8->11：ATT
9->8：RAD
10->11：ATT
11->7：VOB
12->2：WP
13->14：ADV
14->2：COO
15->18：ATT
16->15：RAD
17->18：ATT
18->14：VOB


In [136]:
child_dict_list

[{},
 {'SBV': [0], 'VOB': [3], 'WP': [4, 11], 'COO': [6, 13]},
 {},
 {'ATT': [2]},
 {},
 {},
 {'ADV': [5], 'VOB': [10]},
 {'RAD': [8]},
 {},
 {},
 {'ATT': [7, 9]},
 {},
 {},
 {'ADV': [12], 'VOB': [17]},
 {'RAD': [15]},
 {},
 {},
 {'ATT': [14, 16]}]

In [137]:
def build_parse_child_dict(words, postags, arcs):
    """
    为句子中的每个词语维护一个保存句法依存儿子节点的字典
    Args:
        words: 分词列表
        postags: 词性列表
        arcs: 句法依存列表
    """
    child_dict_list = []
    for index in range(len(words)):
        child_dict = dict()
        for arc_index in range(len(arcs)):
            if arcs[arc_index].head == index + 1:
#                 if child_dict.has_key(arcs[arc_index].relation):
                if (arcs[arc_index].relation) in child_dict:
                    child_dict[arcs[arc_index].relation].append(arc_index)
                else:
                    child_dict[arcs[arc_index].relation] = []
                    child_dict[arcs[arc_index].relation].append(arc_index)
        if ('SBV') in child_dict:
            print(words[index],child_dict['SBV'])
        child_dict_list.append(child_dict)
    return child_dict_list

In [143]:
def complete_e(words, postags, child_dict_list, word_index):
    """
    完善识别的部分实体
    """
    child_dict = child_dict_list[word_index]
    prefix = ''
    if ('ATT') in child_dict:
        for i in range(len(child_dict['ATT'])):
            prefix = prefix+complete_e(words, postags, child_dict_list, child_dict['ATT'][i])
    
    postfix = ''
    if postags[word_index] == 'v':
        if ('VOB') in child_dict:
            postfix =postfix+ complete_e(words, postags, child_dict_list, child_dict['VOB'][0])
        if ('SBV') in child_dict:
            prefix =prefix+ complete_e(words, postags, child_dict_list, child_dict['SBV'][0])

    return prefix + words[word_index] + postfix

In [144]:
index=1
child_dict = child_dict_list[index]
print(child_dict)
e2=complete_e(words, postags, child_dict_list, child_dict['VOB'][0])

{'SBV': [0], 'VOB': [3], 'WP': [4, 11], 'COO': [6, 13]}


In [145]:
for i in filter(None, e2.split(',')):
    print(i)

足球运动员


In [146]:
    for index in range(len(postags)):
        # 抽取以谓词为中心的事实三元组
        if postags[index] == 'v':# 找到谓语
            child_dict = child_dict_list[index]
            print(child_dict)
            # 主谓宾
            if   ('SBV') in child_dict and  ('VOB') in child_dict:
                e1 = complete_e(words, postags, child_dict_list, child_dict['SBV'][0])
                r = words[index]
                e2 = complete_e(words, postags, child_dict_list, child_dict['VOB'][0])
                print("%s\t%s\t%s\n" % (e1, r, e2)) #此处三元组的输出格式有待规范
                # 此部分代码绝密，翻版必究-------------------------------------
                if ('COO') in child_dict:
                    for i in child_dict['COO']:
                        child_dict_list[i]['SBV']=child_dict['SBV']
                #----------------------------------------------------
                # out_file.flush()
            # 定语后置，动宾关系
            
            if arcs[index].relation == 'ATT':
                if  ('VOB') in child_dict:
                    e1 = complete_e(words, postags, child_dict_list, arcs[index].head - 1)
                    r = words[index]
                    e2 = complete_e(words, postags, child_dict_list, child_dict['VOB'][0])
                    temp_string = r+e2
                    if temp_string == e1[:len(temp_string)]:
                        e1 = e1[len(temp_string):]
                    if temp_string not in e1:
                        print("%s\t%s\t%s\n" % (e1, r, e2)) #此处三元组的输出格式有待规范
                        # out_file.flush()
            # 含有介宾关系的主谓动补关系
            if  ('SBV') in child_dict and  ('CMP') in child_dict:
                #e1 = words[child_dict['SBV'][0]]
                e1 = complete_e(words, postags, child_dict_list, child_dict['SBV'][0])
                cmp_index = child_dict['CMP'][0]
                r = words[index] + words[cmp_index]
                if  ('POB') in child_dict_list[cmp_index]:
                    e2 = complete_e(words, postags, child_dict_list, child_dict_list[cmp_index]['POB'][0])
                    print("%s\t%s\t%s\n" % (e1, r, e2)) #此处三元组的输出格式有待规范
                    # out_file.flush()

        # 尝试抽取命名实体有关的三元组
        if netags[index][0] == 'S' or netags[index][0] == 'B':
            ni = index
            if netags[ni][0] == 'B':
                while netags[ni][0] != 'E':
                    ni += 1
                e1 = ''.join(words[index:ni+1])
            else:
                e1 = words[ni]
            if arcs[ni].relation == 'ATT' and postags[arcs[ni].head-1] == 'n' and netags[arcs[ni].head-1] == 'O':
                r = complete_e(words, postags, child_dict_list, arcs[ni].head-1)
                if e1 in r:
                    r = r[(r.index(e1)+len(e1)):]
                if arcs[arcs[ni].head-1].relation == 'ATT' and netags[arcs[arcs[ni].head-1].head-1] != 'O':
                    e2 = complete_e(words, postags, child_dict_list, arcs[arcs[ni].head-1].head-1)
                    mi = arcs[arcs[ni].head-1].head-1
                    li = mi
                    if netags[mi][0] == 'B':
                        while netags[mi][0] != 'E':
                            mi += 1
                        e = ''.join(words[li+1:mi+1])
                        e2 += e
                    if r in e2:
                        e2 = e2[(e2.index(r)+len(r)):]
                    if r+e2 in sentence:
                        print("%s\t%s\t%s\n" % (e1, r, e2)) #此处三元组的输出格式有待规范
                        # out_file.flush()

{'SBV': [0], 'VOB': [3], 'WP': [4, 11], 'COO': [6, 13]}
小明	是	足球运动员

{'ADV': [5], 'VOB': [10], 'SBV': [0]}
小明	是	杰出上海人

{'ADV': [12], 'VOB': [17], 'SBV': [0]}
小明	是	优秀中国人



In [16]:
arc.head

117

In [17]:
arc.relation

'WP'